In [4]:
# Spinglass(tau = infinity)

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
start_seed = 42
start_alpha = 0.0
#seed = start_seed
iter = 1e7
energy_series = []
m_series = []

spins, h, J, st, st1, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)

for i in range(int(iter)):
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st = st + torch.exp(-simulation.calculate_energy(spins, h, J))
    st1 = st1 + torch.exp(-simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = st1 / st
    alpha = min(alpha + alpha_inc, 0.999)
    m = 2 * alpha * (z - 0.5)
    if i % int(iter / 1e6) == 0:
        energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
        m_series.append(m.tolist())

energy_series.insert(0, 'energy')
energy_file_name = f'tauinf_spinglass_annealing_energy'
m_file_name = f'tauinf_spinglass_annealing_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)

In [1]:
# Spinglass(tau = infinity)
# Non-annealing

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
start_seed = 42
start_alpha = 0.0
#seed = start_seed
iter = 1e7
energy_series = []
m_series = []

spins, h, J, st, st1, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)
alpha = 0.999

for i in range(int(iter)):
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st = st + torch.exp(-simulation.calculate_energy(spins, h, J))
    st1 = st1 + torch.exp(-simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = st1 / st
    m = 2 * alpha * (z - 0.5)
    if i % int(iter / 1e6) == 0:
        energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
        m_series.append(m.tolist())

energy_series.insert(0, 'energy')
energy_file_name = f'tauinf_spinglass_nonannealing_energy'
m_file_name = f'tauinf_spinglass_nonannealing_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)

In [88]:
# Homogeneous fully connected ising model(tau=infinity)

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
start_seed = 42
#seed = start_seed
iter = 1e6
energy_series = []
m_series = []

spins, h, J, st, st1, alpha, alpha_inc = simulation.initialize_homogeneous(n, start_seed, iter, device)

for i in range(int(iter)):
    spins = 2 * (simulation.decision_probabilities(st, st1, alpha) > torch.rand(n, device=device)) - 1
    st += torch.exp(-simulation.calculate_energy(spins, h, J))
    st1 += torch.exp(-simulation.calculate_energy(spins, h, J)) * ((spins + 1) / 2)
    z = (st1 / st)
    alpha += alpha_inc
    m = 2 * alpha * (z - 0.5)
    energy_series.append(simulation.calculate_energy(spins, h, J).cpu().item())
    m_series.append(m.tolist())

energy_series.insert(0, 'energy')
energy_file_name = 'tauinf_homo_energy'
m_file_name = 'tauinf_homo_m'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)